In [1]:
import pandas as pd
import numpy as np

In [2]:
import bar_chart_race as bcr

In [3]:
gdp = pd.read_csv('gdp.csv',  encoding = 'unicode_escape')

In [4]:
gdp['Value'] = [x.replace(',', '') for x in gdp['Value']]
gdp['Value'] = [x.replace(':', '') for x in gdp['Value']]
gdp['Value'] = [x.replace('', '0') for x in gdp['Value']]
gdp['Value'].fillna('0',inplace=True)
gdp['Value'] = gdp['Value'].astype(float)

In [5]:
gdp.Value

0        1.000907e+15
1        3.060301e+11
2        6.090008e+09
3        6.090008e+09
4        6.090008e+09
             ...     
17885    9.060503e+07
17886    3.070806e+07
17887    2.020202e+07
17888    2.020600e+07
17889    1.030804e+07
Name: Value, Length: 17890, dtype: float64

In [6]:
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17890 entries, 0 to 17889
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TIME                17890 non-null  int64  
 1   GEO                 17890 non-null  object 
 2   UNIT                17890 non-null  object 
 3   Value               17890 non-null  float64
 4   Flag and Footnotes  2491 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 699.0+ KB


In [7]:
pop = pd.read_csv('popul.csv', encoding = 'unicode_escape')

In [8]:
pop['Value'] = [x.replace(',', '') for x in pop['Value']]
pop['Value'] = [x.replace(':', '') for x in pop['Value']]
pop['Value'] = [x.replace('', '0') for x in pop['Value']]
pop['Value'].fillna('0',inplace=True)
pop['Value'] = pop['Value'].astype(float)

In [9]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17640 entries, 0 to 17639
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TIME                17640 non-null  int64  
 1   GEO                 17640 non-null  object 
 2   UNIT                17640 non-null  object 
 3   Value               17640 non-null  float64
 4   Flag and Footnotes  567 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 689.2+ KB


In [10]:
pop

,TIME,GEO,UNIT,Value,Flag and Footnotes
0,2010,European Union - 27 countries (from 2020),Thousand,4.040009e+11,NaN
1,2010,Belgium,Thousand,1.000809e+09,NaN
2,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Thousand,1.010004e+07,NaN
3,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Thousand,1.010004e+07,NaN
4,2010,Arr. de Bruxelles-Capitale/Arr. Brussel-Hoofdstad,Thousand,1.010004e+07,NaN
...,...,...,...,...,...
17635,2019,"Mardin, Batman, Sirnak, Siirt",Thousand,2.020905e+07,NaN
17636,2019,Mardin,Thousand,8.030301e+05,NaN
17637,2019,Batman,Thousand,6.000301e+05,NaN
17638,2019,Sirnak,Thousand,5.020601e+05,NaN


In [11]:
df = pd.merge(gdp,pop, how='left', left_on=['GEO','TIME'], right_on=['GEO','TIME'])

In [12]:
df

,TIME,GEO,UNIT_x,Value_x,Flag and Footnotes_x,UNIT_y,Value_y,Flag and Footnotes_y
0,2010,European Union - 27 countries (from 2020),Million euro,1.000907e+15,NaN,Thousand,4.040009e+11,NaN
1,2010,Belgium,Million euro,3.060301e+11,NaN,Thousand,1.000809e+09,NaN
2,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Million euro,6.090008e+09,NaN,Thousand,1.010004e+07,NaN
3,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Million euro,6.090008e+09,NaN,Thousand,1.010004e+07,NaN
4,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Million euro,6.090008e+09,NaN,Thousand,1.010004e+07,NaN
...,...,...,...,...,...,...,...,...
21825,2019,"Mardin, Batman, Sirnak, Siirt",Million euro,9.060503e+07,NaN,Thousand,2.020905e+07,NaN
21826,2019,Mardin,Million euro,3.070806e+07,NaN,Thousand,8.030301e+05,NaN
21827,2019,Batman,Million euro,2.020202e+07,NaN,Thousand,6.000301e+05,NaN
21828,2019,Sirnak,Million euro,2.020600e+07,NaN,Thousand,5.020601e+05,NaN


In [13]:
df['GDP/hab']=round((df['Value_x'] *1000) / df['Value_y'],2)

In [14]:
df

,TIME,GEO,UNIT_x,Value_x,Flag and Footnotes_x,UNIT_y,Value_y,Flag and Footnotes_y,GDP/hab
0,2010,European Union - 27 countries (from 2020),Million euro,1.000907e+15,NaN,Thousand,4.040009e+11,NaN,2477487.24
1,2010,Belgium,Million euro,3.060301e+11,NaN,Thousand,1.000809e+09,NaN,305782.71
2,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Million euro,6.090008e+09,NaN,Thousand,1.010004e+07,NaN,602968.71
3,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Million euro,6.090008e+09,NaN,Thousand,1.010004e+07,NaN,602968.71
4,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Million euro,6.090008e+09,NaN,Thousand,1.010004e+07,NaN,602968.71
...,...,...,...,...,...,...,...,...,...
21825,2019,"Mardin, Batman, Sirnak, Siirt",Million euro,9.060503e+07,NaN,Thousand,2.020905e+07,NaN,4483.39
21826,2019,Mardin,Million euro,3.070806e+07,NaN,Thousand,8.030301e+05,NaN,38240.24
21827,2019,Batman,Million euro,2.020202e+07,NaN,Thousand,6.000301e+05,NaN,33668.35
21828,2019,Sirnak,Million euro,2.020600e+07,NaN,Thousand,5.020601e+05,NaN,40246.18


In [15]:
df[df['GEO']=='Galicia']

,TIME,GEO,UNIT_x,Value_x,Flag and Footnotes_x,UNIT_y,Value_y,Flag and Footnotes_y,GDP/hab
764,2010,Galicia,Million euro,5.060706e+09,NaN,Thousand,20707010.08,NaN,244395.79
2947,2011,Galicia,Million euro,5.050706e+09,NaN,Thousand,20707010.07,NaN,243912.86
5130,2012,Galicia,Million euro,5.030907e+09,NaN,Thousand,20706050.07,NaN,242967.97
7313,2013,Galicia,Million euro,5.030902e+09,NaN,Thousand,20705030.02,NaN,242979.70
9496,2014,Galicia,Million euro,5.040109e+09,NaN,Thousand,20703090.03,NaN,243447.19
11679,2015,Galicia,Million euro,5.060606e+09,NaN,Thousand,20702050.01,NaN,244449.51
13862,2016,Galicia,Million euro,5.080208e+09,NaN,Thousand,20701030.00,NaN,245408.47
16045,2017,Galicia,Million euro,6.000309e+09,NaN,Thousand,20700030.04,NaN,289869.58
18228,2018,Galicia,Million euro,6.020403e+09,p,Thousand,20609090.00,p,292123.67
20411,2019,Galicia,Million euro,6.040402e+09,p,Thousand,20609080.09,p,293094.21


In [16]:
clas = pd.read_csv('clasif.csv')
clas

,Value,Type
0,Spain,Country
1,Noroeste (ES),NUT1
2,Galicia,NUT2
3,A Coruña,NUT3
4,Lugo,NUT3
...,...,...
118,Região Autónoma dos Açores (PT),NUT2
119,Região Autónoma dos Açores (PT),NUT3
120,Região Autónoma da Madeira (PT),NUT1
121,Região Autónoma da Madeira (PT),NUT2


In [17]:
df = pd.merge(df,clas, how='left', left_on=['GEO'], right_on=['Value'])

In [18]:
df

,TIME,GEO,UNIT_x,Value_x,Flag and Footnotes_x,UNIT_y,Value_y,Flag and Footnotes_y,GDP/hab,Value,Type
0,2010,European Union - 27 countries (from 2020),Million euro,1.000907e+15,NaN,Thousand,4.040009e+11,NaN,2477487.24,European Union - 27 countries (from 2020),Country
1,2010,Belgium,Million euro,3.060301e+11,NaN,Thousand,1.000809e+09,NaN,305782.71,NaN,NaN
2,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Million euro,6.090008e+09,NaN,Thousand,1.010004e+07,NaN,602968.71,NaN,NaN
3,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Million euro,6.090008e+09,NaN,Thousand,1.010004e+07,NaN,602968.71,NaN,NaN
4,2010,Région de Bruxelles-Capitale/Brussels Hoofdste...,Million euro,6.090008e+09,NaN,Thousand,1.010004e+07,NaN,602968.71,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
22425,2019,"Mardin, Batman, Sirnak, Siirt",Million euro,9.060503e+07,NaN,Thousand,2.020905e+07,NaN,4483.39,NaN,NaN
22426,2019,Mardin,Million euro,3.070806e+07,NaN,Thousand,8.030301e+05,NaN,38240.24,NaN,NaN
22427,2019,Batman,Million euro,2.020202e+07,NaN,Thousand,6.000301e+05,NaN,33668.35,NaN,NaN
22428,2019,Sirnak,Million euro,2.020600e+07,NaN,Thousand,5.020601e+05,NaN,40246.18,NaN,NaN


In [19]:
sub = df[df['Type'].isin(['Country','NUT2'])].copy()

In [20]:
sub

,TIME,GEO,UNIT_x,Value_x,Flag and Footnotes_x,UNIT_y,Value_y,Flag and Footnotes_y,GDP/hab,Value,Type
0,2010,European Union - 27 countries (from 2020),Million euro,1.000907e+15,NaN,Thousand,4.040009e+11,NaN,2477487.24,European Union - 27 countries (from 2020),Country
762,2010,Spain,Million euro,1.000702e+13,NaN,Thousand,4.060506e+09,NaN,2464476.26,Spain,Country
764,2010,Galicia,Million euro,5.060706e+09,NaN,Thousand,2.070701e+07,NaN,244395.79,Galicia,NUT2
780,2010,País Vasco,Million euro,6.040608e+09,NaN,Thousand,2.010801e+07,NaN,300408.05,País Vasco,NUT2
784,2010,Comunidad Foral de Navarra,Million euro,1.070907e+09,NaN,Thousand,6.030500e+05,NaN,1775817.98,Comunidad Foral de Navarra,NUT2
...,...,...,...,...,...,...,...,...,...,...,...
21944,2019,Região Autónoma da Madeira (PT),Million euro,5.000609e+07,NaN,Thousand,2.050400e+05,NaN,243884.55,Região Autónoma da Madeira (PT),NUT2
21947,2019,Região Autónoma da Madeira (PT),Million euro,5.000609e+07,NaN,Thousand,0.000000e+00,NaN,inf,Região Autónoma da Madeira (PT),NUT2
21950,2019,Região Autónoma da Madeira (PT),Million euro,5.000609e+07,NaN,Thousand,2.050400e+05,NaN,243884.55,Região Autónoma da Madeira (PT),NUT2
21953,2019,Região Autónoma da Madeira (PT),Million euro,5.000609e+07,NaN,Thousand,2.050400e+05,NaN,243884.55,Região Autónoma da Madeira (PT),NUT2


In [21]:
# Replace infinite updated data with nan
sub.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN
sub.dropna(subset=['GDP/hab'],inplace=True)


In [22]:
sub.drop_duplicates(inplace=True)

In [23]:
sub = sub[['GEO','TIME','GDP/hab']]
sub

,GEO,TIME,GDP/hab
0,European Union - 27 countries (from 2020),2010,2477487.24
762,Spain,2010,2464476.26
764,Galicia,2010,244395.79
780,País Vasco,2010,300408.05
784,Comunidad Foral de Navarra,2010,1775817.98
...,...,...,...
21881,Centro (PT),2019,153007.27
21890,Área Metropolitana de Lisboa,2019,339374.79
21898,Alentejo,2019,1471894.98
21905,Região Autónoma dos Açores (PT),2019,198049.57


In [24]:
df = sub.pivot_table(values = 'GDP/hab',index = ['TIME'],columns = 'GEO')
df

GEO,Alentejo,Algarve,Andalucía,Aragón,Canarias,Castilla y León,Castilla-la Mancha,Cataluña,Centro (PT),Ciudad de Ceuta,...,Illes Balears,La Rioja,Norte,País Vasco,Portugal,Região Autónoma da Madeira (PT),Região Autónoma dos Açores (PT),Región de Murcia,Spain,Área Metropolitana de Lisboa
TIME,,,,,,,,,,,,,,,,,,,,,
2010,1431728.04,174485.43,1295649.80,294137.54,199989.75,245849.33,153965.72,2840775.79,149253.96,1313375.15,...,205884.16,235501.180,163370.03,300408.05,107036.33,196064.46,150490.24,199028.26,2464476.26,291845.99
2011,1433215.54,173992.92,1295439.66,294089.12,199955.07,245830.31,153243.81,1547246.15,148783.45,1310091.29,...,205844.62,234760.670,133647.07,300382.48,107005.96,196078.80,150455.78,198943.02,2464112.88,291332.20
2012,1419701.83,173508.32,1283557.82,292207.49,154440.35,244892.64,152970.88,1548528.04,148300.23,1306221.88,...,203059.30,233784.945,133312.51,299408.45,106027.09,194175.65,149488.34,198010.58,2463367.17,289419.11
2013,1421618.92,173759.46,1283308.19,293118.15,153730.88,244413.92,152943.84,1548490.90,148988.52,1306585.17,...,203039.20,234182.790,133648.17,298930.82,106957.08,194660.84,149973.11,198011.15,2463238.04,289452.39
2014,1436687.16,174500.79,1283426.54,293164.88,153733.33,245516.01,152468.98,1551012.91,149472.96,1306460.41,...,203989.91,234547.060,163405.65,299442.56,106987.21,195145.73,149995.28,198029.70,2463552.28,289907.73
2015,1439871.39,175262.96,1295503.62,293225.02,199009.66,246505.60,153466.38,2845156.87,150000.10,1307088.90,...,205918.18,235552.460,164069.92,300452.60,107058.04,196523.24,150946.00,199923.22,2464552.27,290409.55
2016,1453116.07,199252.63,1294531.85,295134.77,199960.06,247044.51,153984.02,2855364.68,150962.83,1318777.08,...,206927.63,265766.470,165524.94,301904.26,108027.78,197059.54,151467.89,200873.54,2487691.02,291374.63
2017,1456107.34,223272.32,1306595.53,296144.86,200944.69,248014.78,154517.04,2869262.22,151501.83,1317515.53,...,297900.46,266457.495,166205.75,302923.70,109018.20,198536.96,196534.94,288315.02,2488856.86,336519.64
2018,1471273.24,225016.96,1318415.70,297135.15,201451.46,249027.35,200495.03,2866486.04,152480.07,1331079.63,...,298871.19,267454.030,196705.42,349131.94,200008.50,199492.77,197069.23,289236.38,2511890.80,337944.63


In [25]:
import bar_chart_race as bcr

In [26]:
bcr.bar_chart_race(df = df, 
                   n_bars = 20, 
                   period_length=500,
                   sort='desc',
                   title='GDP per capita 2010-2019',
                   filename = 'gdp.mp4')


C:\Users\SergioPer\Miniconda3\envs\data_processing\lib\site-packages\bar_chart_race\_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
C:\Users\SergioPer\Miniconda3\envs\data_processing\lib\site-packages\bar_chart_race\_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
C:\Users\SergioPer\Miniconda3\envs\data_processing\lib\site-packages\bar_chart_race\_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "
